In [4]:
import pyspark
from pyspark.sql import SparkSession
import numpy as np


# Preprocessing 

In [5]:
#create py_spark context
spark = SparkSession.builder.appName("Amazon Reviews").\
config("spark.memory.offHeap.enabled","true").\
config("spark.memory.offHeap.size","10g").\
getOrCreate()

24/04/08 11:19:14 WARN Utils: Your hostname, Bryan resolves to a loopback address: 127.0.1.1; using 172.20.218.41 instead (on interface eth0)
24/04/08 11:19:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/08 11:19:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
#import data
df = spark.read.csv('data/amazon_reviews.csv', header = True)

In [17]:
df.show(5,0)

+---+------------+-------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------+-----------+----------+----------+------------------+--------------------+------------------+
|_c0|reviewerName|overall|reviewText                                                                                                                                                                                                                                                                                                                                          |reviewTime|day_diff|helpful_yes|helpful_no|total_vote|score_pos_neg_diff|score_average_rating|wilson_lower_bound|
+---+------------+-------+------------

24/04/08 11:35:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , reviewerName, overall, reviewText, reviewTime, day_diff, helpful_yes, helpful_no, total_vote, score_pos_neg_diff, score_average_rating, wilson_lower_bound
 Schema: _c0, reviewerName, overall, reviewText, reviewTime, day_diff, helpful_yes, helpful_no, total_vote, score_pos_neg_diff, score_average_rating, wilson_lower_bound
Expected: _c0 but found: 
CSV file: file:///home/bryanhurtado/projects/Sentiment_analysis_amazon_reviews/data/amazon_reviews.csv


In [18]:
#count the total of information
df.count()

4915

In [20]:
#review columns and types
df.columns

['_c0',
 'reviewerName',
 'overall',
 'reviewText',
 'reviewTime',
 'day_diff',
 'helpful_yes',
 'helpful_no',
 'total_vote',
 'score_pos_neg_diff',
 'score_average_rating',
 'wilson_lower_bound']

In [21]:
len(df.columns)

12

In [22]:
df.dtypes

[('_c0', 'string'),
 ('reviewerName', 'string'),
 ('overall', 'string'),
 ('reviewText', 'string'),
 ('reviewTime', 'string'),
 ('day_diff', 'string'),
 ('helpful_yes', 'string'),
 ('helpful_no', 'string'),
 ('total_vote', 'string'),
 ('score_pos_neg_diff', 'string'),
 ('score_average_rating', 'string'),
 ('wilson_lower_bound', 'string')]

In [ ]:
#delete columns which are not neccesary
columns_to_drop = ["",""]
df = df.drop(*columns_to_drop)

## Review scores

## Review Texts